In [30]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

from bokeh.io import show, output_notebook
from bokeh.models import GeoJSONDataSource

output_notebook()

Loading BokehJS ...

Data:

In [19]:
## GIS from Ron
lccTrees = gpd.read_file("/home/daniel/Documents/grants/"
                    "geneClust/data/LCCdata/modifiedGISforApp"
                    "/LCCtrees.geojson")

lccSensors = gpd.read_file("/home/daniel/Documents/grants/"
                    "geneClust/data/LCCdata/modifiedGISforApp"
                    "/LCCSensors.geojson")

##  clear up the LI/WLI difference
lccSensors['BAUMART'] = lccSensors.BAUMART.str.replace('LI','WLI')

## make the unique IDs compatible:
lccTrees['Tree_ID'] = lccTrees['Tree_ID'].str.replace('K','')

The Herrmann research group is now going to focus on Ash, Oak, and Linden. Most likely candidates for sampling are trees that (1) have sensors and (2) were previously sampled. 

In [ ]:
## get the trees that were previously sampled by Herrmann et al. (2020)
## we could narrow down to the species that were sampled previously:

herrmannOldTrees = [
                "129.0",
                "317.0",
                "733.0",
                "33.0" ,
                "513.1",
                "754.0",
                "444.0",
                "455.0",
                "517.1"
                ]

## subset to these:
oldTreesFilt = lccTrees.Tree_ID.isin(herrmannOldTrees)
oldNewTrees = lccTrees[oldTreesFilt]

In [41]:
## did all of these have sensors?
sensorFilt = oldNewTrees.Tree_ID.isin(lccSensors.Name)
## nope, two do not. They are?
oldNewTrees[~sensorFilt][['Tree_ID', 'spec']]

,Tree_ID,spec
546,513.1,BAH
551,517.1,WLI


Take a look at these:

In [27]:
## color by species 
colorDic = {'BAH':'#f19c43',
            'GES':'#861c83',
            'SEI':'#549ad7',
            'WLI':'#3c7403'}
colsSpec = oldNewTrees.spec.apply(lambda x: colorDic[x])

##markersize by girth
BHDmarker = (oldNewTrees.BHD_2020
                .str.replace(',','.')
                .astype('float64'))

In [54]:
oldNewTrees.Tree_ID

34      33.0
143    129.0
335    317.0
473    444.0
484    455.0
546    513.1
551    517.1
776    733.0
798    754.0
Name: Tree_ID, dtype: object

In [53]:
oldNewTrees.Tree_ID.iloc[0]

'33.0'

In [55]:
fig, ax = plt.subplots(figsize=(15,7.5))
oldNewTrees.plot(c=colsSpec, ax=ax, markersize = BHDmarker)

z=0
for x,y in enumerate(zip(oldNewTrees['E_UTM'].to_list(), oldNewTrees['N_UTM'].to_list())):
    ax.text(x,y,oldNewTrees.Tree_ID.iloc[z])
    z += 1
    
    #ax.text(oldNewTrees['E_UTM'].to_list(), 
    #    oldNewTrees['N_UTM'].to_list(), 
    #    oldNewTrees['Tree_ID'].to_list())

TypeError: float() argument must be a string or a number, not 'tuple'

<Figure size 1080x540 with 1 Axes>

In [32]:
## how would we do a bokeh plot of these?

geosource = GeoJSONDataSource(geojson = oldNewTrees.to_json())

In [ ]:
p = figure (title='Trees previously sampled ',
           plot_height=600,
            plot_width=950,
            toolbar_location='below',
            tools='pan, wheel_zoom, box_zoom, reset'
           )

In [ ]:
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

states = p.patches('xs', 'ys', source=geosource,
                  fill_color=None,
                  line_color='Gray',
                  line_width=0.25,
                  fill_alpha=1)

##hover tool
p.add_tools(HoverTool(renderers = [states],
                     tooltips = [('State', '@NAME'), ('Population', '@POPESTIMATE2018')]))

show(p)

In [14]:
## remove maples, which they will not sample in this project. 

acerFilt = (oldNewTrees.spec != 'BAH')
oldNewTrees = oldNewTrees[acerFilt]

In [15]:
oldNewTrees

,FID_,Tree_ID,spec,N_UTM,E_UTM,Elevation,BHD_2020,Zuwachs_20,hKA_2020,hG_2020,...,3_TRO_2020,4_FAEULE_3,5_PILZKO_1,6_PILZso_1,7_ETS_2020,8_Schael_1,refTree_ID,dist_refTr,ang_refTre,geometry
143,144.0,129.0,SEI,5693875.49,312672.65,102.51528,"133,7","-0,3","10,7","35,8",...,30,40,50,60,na,81,na,na,na,POINT (312672.650 5693875.490)
335,336.0,317.0,SEI,5693950.54,312702.11,102.33131,"86,5","6,0","15,3","37,0",...,30,40,50,60,na,80,na,na,na,POINT (312702.110 5693950.540)
473,474.0,444.0,WLI,5693981.77,312721.43,102.82352,"81,5","0,0","14,9","29,2",...,30,40,50,60,na,80,na,na,na,POINT (312721.430 5693981.770)
484,485.0,455.0,WLI,5694002.68,312725.09,102.57285,"69,6","0,6","8,5","34,0",...,30,40,50,60,na,80,na,na,na,POINT (312725.090 5694002.680)
551,552.0,517.1,WLI,5693987.69,312736.89,102.60938,"79,9","0,7","11,4","30,9",...,30,40,50,60,na,80,na,na,na,POINT (312736.890 5693987.690)
776,777.0,733.0,SEI,5693866.03,312730.01,102.64779,"53,6","1,6","16,3","30,0",...,30,40,50,60,na,80,na,na,na,POINT (312730.010 5693866.030)
